Picking images containing monolayers and grouping them into a separate folder

In [11]:
%config InlineBackend.figure_format='retina'
%matplotlib inline

import numpy as np

import matplotlib.pyplot as plt
import cv2

from PIL import Image
import os


x = np.loadtxt('C:/Users/Evgeniy/Desktop/Advanced computing/flakes/monolayer_labels.txt', delimiter=',')
lb=np.int16(x)
k=0


for j in range (0,3025,1):
    if k< len(lb) and j==lb[k]:
        im = Image.open('C:/Users/Evgeniy/Desktop/Advanced computing/flakes/treated/tr'+str(j)+'.jpg')    
        im.save('C:/Users/Evgeniy/Desktop/Advanced computing/flakes/monolayers/tr'+str(j)+'.jpg')
        k=k+1

In order to use a balanced data set containing equal amount of images with and without monolayers I pick random images not labeled as monolayer-containing:

In [13]:
import random
k=0
mlist=[]
m=0
while k < len(lb):
    m=random.randint(1,3024)
    if m not in lb and m not in mlist: 
        k=k+1
        im = Image.open('C:/Users/Evgeniy/Desktop/Advanced computing/flakes/treated/tr'+str(m)+'.jpg')    
        im.save('C:/Users/Evgeniy/Desktop/Advanced computing/flakes/not_monolaeyrs/tr'+str(m)+'.jpg')
        mlist.append(m)

In [14]:
from keras import applications
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
import numpy as np


# for example load the VGG16 network
model = applications.VGG16(include_top=False,
                           weights='imagenet')

C:\Users\Evgeniy\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Calculating features using pre-trained VGG 16 CNN:

In [15]:
import os

def compute_features(directory):
    features = []
    fl_imgs = os.listdir(directory)
    for flake in fl_imgs:
        img_path = directory + flake
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features_ = model.predict(x)
        features.append(features_.ravel())
        
    return features

ml_features = compute_features('C:/Users/Evgeniy/Desktop/Advanced computing/flakes/monolayers/')
non_ml_features = compute_features('C:/Users/Evgeniy/Desktop/Advanced computing/flakes/not_monolaeyrs/')

In [17]:
X = ml_features + non_ml_features
X = np.array(X)
y = [1] * len(ml_features) + [0] * len(non_ml_features)

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

Implementing logistic regression:

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
lr = LogisticRegression()

grid = GridSearchCV(lr, {'C': [1e-010, 1e-08, 1e-06, 1e-04, 1e-02, 0.1]}, cv=7)
grid.fit(X_train, y_train)

C:\Users\Evgeniy\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Evgeniy\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


GridSearchCV(cv=7, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [1e-10, 1e-08, 1e-06, 0.0001, 0.01, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [20]:
grid.best_params_

{'C': 0.0001}

In [21]:
grid.score(X_test, y_test)

0.8979591836734694

Implementing random forest classifier:

In [22]:
from sklearn.ensemble import RandomForestClassifier


RFC=RandomForestClassifier()
grid = GridSearchCV(RFC, {'n_estimators': [20,50,100,200,500] }, cv=7)
grid.fit(X_train, y_train)
 

GridSearchCV(cv=7, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [20, 50, 100, 200, 500]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [23]:
grid.best_params_

{'n_estimators': 200}

In [24]:
grid.score(X_test, y_test)

0.8571428571428571

Combination of pre-trained VGG 16 as feature extractor and linear regression yields values of accuracy of about 0.9 which looks quite reasonable. That shows the possibility of implementing CNN trained on some 'conventional' images for some specific research applications as flake thickness recognition presented here. 

The implementation discussed here is the most simple ground-level problem (whether a monolayer is present or not). The more useful application can be determination of exact thickness of presented flakes (2,3...10 layers) which can be viewed as a next step.    